## 簡介
* 將常用的 decorator 紀錄於此
* function 可以當作被傳遞的物件，decorator 善用了此項特點
* decorator 用途與特性：
    * 簡潔：把不同 function 相同內容抽出為 decorator，以修飾 function
    * 有序：不同 decorator 使用在同一 function 的順序會導致結果有別 


## 學習資源
* [部落格：Python進階技巧 (3) — 神奇又美好的 Decorator ，嗷嗚！](https://medium.com/citycoddee/python%E9%80%B2%E9%9A%8E%E6%8A%80%E5%B7%A7-3-%E7%A5%9E%E5%A5%87%E5%8F%88%E7%BE%8E%E5%A5%BD%E7%9A%84-decorator-%E5%97%B7%E5%97%9A-6559edc87bc0)

In [39]:
"""
EX：function 可以當作被傳遞的物件
* aa 是 info() return 的物件，該物件又是 print_something() 
* 故在呼叫 aa() 時，等同於在呼叫 print_something()
* 並且 print_something() 捕捉了 name 這個 variable，稱 name 為 captured variable
* 帶有 captured variable 的 function 稱為 closure function
* 若要做操作 captured variable 需變成 nonlocal，如 info2()
"""
def info():
    name = 'user'
    def print_something():
        print(f'hello {name}')
    
    return print_something

aa = info()
print(aa)
aa()

def info2():
    name = 'user'
    def print_something(): 
        nonlocal name
        name += '!'
        print(f'hello {name}')
    
    return print_something

bb = info2()
bb()
bb()
bb()

<function info.<locals>.print_something at 0x0000016550FE2E18>
hello user
hello user!
hello user!!
hello user!!!


In [19]:
"""
EX1：把不同 function 相同內容抽出為 decorator，以修飾 function，如印出 function 名稱、時間做修飾
* 此處的 func 為所有想計算運算時間的 function，如 adder 及 multiplier
* timing 把 func 傳入
* wrap_func 將欲添加的功能加以修飾 func
* timing 回傳 wrap_func 以回傳修飾過的 func
"""
from time import time


def timing(func):
    # This function shows the execution time of
    # the function object passed
    def wrap_func(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        spent_time = end_time - start_time
        print(f' {func.__name__!r} executed in {spent_time:.4f}s')
#         print(f'[{func.__name__}] : {spent_time:.4f}s')
#         print(f'Function {func.__doc__!r} executed in {spent_time:.4f}s')
#         print(f'Function {func.__doc__} executed in {spent_time:.4f}s')
        return result
    return wrap_func

In [13]:
"""
EX1-1
用 function 呼叫
"""
def adder():
    cnt = 0
    for i in range(10):
        cnt += i
    return cnt

def multiplier():
    cnt = 0
    for i in range(10):
        cnt *= i

cnt = timing(adder)()
print(cnt)
cnt = timing(multiplier)()
print(cnt)

 'adder' executed in 0.0000s
45
 'multiplier' executed in 0.0000s
None


In [14]:
"""
EX1-2
用 @ 讓語法簡化
"""
@timing
def adder():
    cnt = 0
    for i in range(10):
        cnt += i
    return cnt

@timing
def multiplier():
    cnt = 0
    for i in range(10):
        cnt *= i
cnt = adder()
multiplier()

 'adder' executed in 0.0000s
 'multiplier' executed in 0.0000s


In [15]:
"""
EX2：decorator 有順序性，如將印出 function 名稱、時間分開做兩個 decorator，越靠近欲修飾的 function 會越先合併
* EX2-1：先合併 print_time 並回吐 wrap_func > print_name 傳入的 func 為 wrap_func > 故印出 function 名字為 wrap_func
* EX2-2：先合併 print_name 並回吐 wrap_func > print_time 傳入的 func 為 wrap_func > wrap_func 傳入 adder 印出其運算時間
"""
def print_name(func):
    # This function shows the execution time of
    # the function object passed
    def wrap_func(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__!r}')
        return result
    return wrap_func

def print_time(func):
    # This function shows the execution time of
    # the function object passed
    def wrap_func(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        spent_time = end_time - start_time
        print(f'{spent_time:.4f}s')
        return result
    return wrap_func

In [17]:
"""
EX2-1
"""
@print_name
@print_time
def adder():
    cnt = 0
    for i in range(10):
        cnt += i
adder()

0.0000s
'wrap_func'


In [18]:
"""
EX2-2
"""
@print_time
@print_name
def adder():
    cnt = 0
    for i in range(10):
        cnt += i
adder()

'adder'
0.0000s
